In [0]:
#text파일 읽기
text_file = sc.textFile("/FileStore/tables/test1.txt")

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/City-Data.parquet/,City-Data.parquet/,0
dbfs:/FileStore/tables/Crime-Data-Boston-2016-1.parquet/,Crime-Data-Boston-2016-1.parquet/,0
dbfs:/FileStore/tables/Crime-Data-Boston-2016.parquet/,Crime-Data-Boston-2016.parquet/,0
dbfs:/FileStore/tables/Crime-Data-Chicago-2016.parquet/,Crime-Data-Chicago-2016.parquet/,0
dbfs:/FileStore/tables/Crime-Data-New-York-1-2016.parquet/,Crime-Data-New-York-1-2016.parquet/,0
dbfs:/FileStore/tables/Crime-Data-New-York-2016.parquet/,Crime-Data-New-York-2016.parquet/,0
dbfs:/FileStore/tables/count.txt/,count.txt/,0
dbfs:/FileStore/tables/databricks_blog-1.json,databricks_blog-1.json,3052424
dbfs:/FileStore/tables/databricks_blog-2.json,databricks_blog-2.json,3052424
dbfs:/FileStore/tables/databricks_blog.json,databricks_blog.json,3052424


In [0]:
text_file.collect()

Out[3]: ['Apache Spark Examples',
 'These examples give a quick overview of the Spark API.',
 'This is Spark time Learn Spark',
 'This is a word count example',
 'that is another test']

In [0]:
#page rank program
def computeContribs(urls,rank):
  num_urls = len(urls)
  for url in urls:
    yield (url, rank / num_urls)

In [0]:
map_link = sc.parallelize([['MapR','Baidu'], ['MapR','Blogger'], ['Baidu','MapR'], ['Blogger','Baidu'],['Blogger', 'Google'],['Google','MapR']])

links = map_link.groupByKey()

#있는 단어에 대하여 초기 rank설정
ranks = links.map(lambda pair: (pair[0],1))
print('initial ranks:', ranks.collect())

for i in range(5):
  cvalues = links.join(ranks)
  contribs = cvalues.flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
  ranks = contribs.reduceByKey(lambda x,y : x + y).mapValues(lambda rank : 0.15+0.85*rank)
  print("ranks(" , i , "):",ranks.collect())
  print

initial ranks: [('Google', 1), ('MapR', 1), ('Baidu', 1), ('Blogger', 1)]
ranks( 0 ): [('Google', 0.575), ('MapR', 1.8499999999999999), ('Baidu', 1.0), ('Blogger', 0.575)]
ranks( 1 ): [('MapR', 1.4887499999999998), ('Blogger', 0.9362499999999999), ('Baidu', 1.1806249999999998), ('Google', 0.394375)]
ranks( 2 ): [('Google', 0.5479062499999999), ('MapR', 1.4887499999999996), ('Baidu', 1.1806249999999998), ('Blogger', 0.7827187499999999)]
ranks( 3 ): [('MapR', 1.6192515624999995), ('Baidu', 1.1153742187499998), ('Blogger', 0.7827187499999998), ('Google', 0.48265546875)]
ranks( 4 ): [('MapR', 1.5083252343749995), ('Google', 0.48265546874999987), ('Blogger', 0.8381819140624998), ('Baidu', 1.1708373828124996)]

In [0]:
#DataFrame 시작
from pyspark.sql import Row
l = [('Ankit',25), ('Jalfaizy',22), ('saurabh',20), ('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x : Row(name=x[0], age = int(x[1])))
print(people.collect())
#rdd에서 dataframe만들기
schemaPeople = sqlContext.createDataFrame(people)
print(type(schemaPeople))
schemaPeople.printSchema()
print(schemaPeople.collect())
print(schemaPeople.count())

[Row(name='Ankit', age=25), Row(name='Jalfaizy', age=22), Row(name='saurabh', age=20), Row(name='Bala', age=26)]
<class 'pyspark.sql.dataframe.DataFrame'>
root
-- name: string (nullable = true)
-- age: long (nullable = true)

[Row(name='Ankit', age=25), Row(name='Jalfaizy', age=22), Row(name='saurabh', age=20), Row(name='Bala', age=26)]
4

In [0]:
#parquet파일 읽기
peopleDF = spark.read.parquet("/FileStore/tables/test/people-10m.parquet")
peopleDF.printSchema()

root
-- id: integer (nullable = true)
-- firstName: string (nullable = true)
-- middleName: string (nullable = true)
-- lastName: string (nullable = true)
-- gender: string (nullable = true)
-- birthDate: timestamp (nullable = true)
-- ssn: string (nullable = true)
-- salary: integer (nullable = true)

In [0]:
#dataFrame 다루기
peopleDF.show(5)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 2| An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 3| Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
 4| Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727|
 5| Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 5 rows

In [0]:
peopleDF.head(5)

Out[18]: [Row(id=1, firstName='Pennie', middleName='Carry', lastName='Hirschmann', gender='F', birthDate=datetime.datetime(1955, 7, 2, 4, 0), ssn='981-43-9345', salary=56172),
 Row(id=2, firstName='An', middleName='Amira', lastName='Cowper', gender='F', birthDate=datetime.datetime(1992, 2, 8, 5, 0), ssn='978-97-8086', salary=40203),
 Row(id=3, firstName='Quyen', middleName='Marlen', lastName='Dome', gender='F', birthDate=datetime.datetime(1970, 10, 11, 4, 0), ssn='957-57-8246', salary=53417),
 Row(id=4, firstName='Coralie', middleName='Antonina', lastName='Marshal', gender='F', birthDate=datetime.datetime(1990, 4, 11, 4, 0), ssn='963-39-4885', salary=94727),
 Row(id=5, firstName='Terrie', middleName='Wava', lastName='Bonar', gender='F', birthDate=datetime.datetime(1980, 1, 16, 5, 0), ssn='964-49-8051', salary=79908)]

In [0]:
print(peopleDF.count())
print(peopleDF.columns)

10000000
['id', 'firstName', 'middleName', 'lastName', 'gender', 'birthDate', 'ssn', 'salary']

In [0]:
peopleDF.select('firstName','birthDate').show(5)

+---------+-------------------+
firstName| birthDate|
+---------+-------------------+
 Pennie|1955-07-02 04:00:00|
 An|1992-02-08 05:00:00|
 Quyen|1970-10-11 04:00:00|
 Coralie|1990-04-11 04:00:00|
 Terrie|1980-01-16 05:00:00|
+---------+-------------------+
only showing top 5 rows

In [0]:
peopleDF.select('gender').distinct().count()

Out[23]: 2

In [0]:
#dataframe에 접근하기 위한 여러 방법들
print(peopleDF.filter(peopleDF['gender'] == 'F').count())

print(peopleDF.filter(peopleDF.gender == 'F').count())

print(peopleDF.filter("gender == 'F'").count())

5187302
5187302
5187302

In [0]:
#다양한 dataframe sql 함수들
from pyspark.sql.functions import year
peopleDF.filter(year('birthDate') > '1995').count()

Out[7]: 849353

In [0]:
#select - 값 가져오기, filter - 조건에 맞는 것 가져오기
test1 = peopleDF.select('firstName').filter("gender = 'M'" and year("birthDate") > "1960")
test1.show(5)

+---------+
firstName|
+---------+
 An|
 Quyen|
 Coralie|
 Terrie|
 Chassidy|
+---------+
only showing top 5 rows

In [0]:
peopleDF.select('firstName',year('birthDate')).filter("gender = 'M'" and year("birthDate") > "1960").show(10)

+---------+---------------+
firstName|year(birthDate)|
+---------+---------------+
 An| 1992|
 Quyen| 1970|
 Coralie| 1990|
 Terrie| 1980|
 Chassidy| 1990|
 Geri| 1970|
 Patria| 1985|
 Terese| 1967|
 Wava| 1963|
 Sophie| 1979|
+---------+---------------+
only showing top 10 rows

In [0]:
#geoupBy절
peopleDF.groupBy("gender").count().show()

+------+-------+
gender| count|
+------+-------+
 F|5187302|
 M|4812698|
+------+-------+

In [0]:
#orderBy절
peopleDF.orderBy("firstName").groupBy("firstName").count().show(5)

+---------+-----+
firstName|count|
+---------+-----+
 Alayna| 1313|
 Melaine| 1237|
 Faye| 1350|
 Tegan| 1222|
 Lashanda| 1282|
+---------+-----+
only showing top 5 rows

In [0]:
peopleDF.groupBy("firstName").count().orderBy("count",ascending = False).show(5)

+---------+-----+
firstName|count|
+---------+-----+
 Sammie| 5462|
 Daryl| 5442|
 Bobbie| 5432|
 Lee| 5411|
 Troy| 5408|
+---------+-----+
only showing top 5 rows

In [0]:
peopleDF.groupBy("firstName").count().orderBy("count").show(5)

+---------+-----+
firstName|count|
+---------+-----+
 Marylynn| 1150|
 Verna| 1154|
 Carlena| 1159|
 Tennie| 1162|
 Thi| 1162|
+---------+-----+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import col

#count는 원래 컬럼이 아니기에 col을 사용
peopleDF.groupBy("firstName").count().filter(col('count') > "5430").orderBy("count", ascending = False).show(5)

+---------+-----+
firstName|count|
+---------+-----+
 Sammie| 5462|
 Daryl| 5442|
 Bobbie| 5432|
+---------+-----+

In [0]:
peopleDF.select('firstName', 'lastName').filter(peopleDF.gender == 'M').show(5)

+---------+--------+
firstName|lastName|
+---------+--------+
 Jasper|Neubigin|
 Robert| Kipling|
 Abdul| Pachta|
 Earle|Gieraths|
 Mickey| Wotton|
+---------+--------+
only showing top 5 rows

In [0]:
testDF1 = peopleDF.select('firstName', 'lastName','birthDate').filter(peopleDF['gender'] == 'M')

In [0]:
testDF2 = testDF1.filter(year('birthDate') > "1990")
#display함수 사용
display(testDF2)

firstName,lastName,birthDate
Theron,Aplin,1995-03-16T05:00:00.000+0000
Valentin,Yakunin,1999-04-06T04:00:00.000+0000
Emmett,Grgic,1991-04-22T04:00:00.000+0000
Nelson,Kelf,1995-08-17T04:00:00.000+0000
Sonny,Kornousek,1994-11-04T05:00:00.000+0000
Lucio,Orpwood,1993-10-28T04:00:00.000+0000
Burl,Fronks,1996-04-20T04:00:00.000+0000
Marco,Mueller,1998-12-23T05:00:00.000+0000
Loyd,Gerish,1998-09-27T04:00:00.000+0000
Marco,Jewett,1993-01-12T05:00:00.000+0000


In [0]:
marysDF = peopleDF.\
          select(year('birthDate').alias('birthYear')).\
          filter("firstName == 'Mary'").\
          filter("gender == 'F'").\
          groupBy('birthYear').\
          count().\
          orderBy('birthYear')

display(marysDF)

birthYear,count
1952,27
1953,25
1954,15
1955,23
1956,28
1957,29
1958,26
1959,28
1960,37
1961,29


In [0]:
dordonDF = peopleDF.\
          select(year("birthDate").alias("birthYear"), "firstName").\
          filter((col("firstName") == 'Donna') | (col("firstName") == 'Dorothy')).\
          filter("gender == 'F'").\
          groupBy('birthYear','firstName').\
          count().\
          orderBy('birthYear','firstName').\
          show(5)

+---------+---------+-----+
birthYear|firstName|count|
+---------+---------+-----+
 1952| Donna| 24|
 1952| Dorothy| 21|
 1953| Donna| 37|
 1953| Dorothy| 38|
 1954| Donna| 32|
+---------+---------+-----+
only showing top 5 rows

In [0]:
dordonDF = peopleDF.\
          select(year("birthDate").alias("birthYear"), "firstName").\
          filter((col("firstName") == 'Donna') | (col("firstName") == 'Dorothy')).\
          filter("gender == 'F'").\
          filter(year("birthDate") > 1990).\
          groupBy('birthYear','firstName').\
          count().\
          orderBy('birthYear','firstName').\
display(dordonDF)

birthYear,firstName,count
1991,Donna,27
1991,Dorothy,25
1992,Donna,30
1992,Dorothy,39
1993,Donna,22
1993,Dorothy,28
1994,Donna,26
1994,Dorothy,21
1995,Donna,33
1995,Dorothy,26


In [0]:
from pyspark.sql.functions import count, desc

top10FemaleFirstNamesDF = (peopleDF
                          .select('firstName')
                          .filter("gender == 'F'")
                          .groupBy("firstName")
                          .agg(count("firstname").alias("total"))
                          .orderBy(desc("total"))
                          .limit(10)
                          )